In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import spacy
import requests
from bs4 import BeautifulSoup

In [4]:
# function for downloading content from the web
def contentDownloader(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html5lib")
    for item in soup.find_all("div", {"class": "postArticle-content js-postField js-notesSource js-trackedPost"}):
        return item.text